# core

> The core contents and layout of the blog application.

In [1]:
# | default_exp core

In [ ]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *
from monsterui.all import *

In [ ]:
#| export
hdrs = (
        *Theme.slate.headers(highlightjs=True),
        Link(rel="icon", href="/static/favicon.png"),
        Script(src="https://unpkg.com/hyperscript.org@0.9.12"),
        Script(src="/static/scripts.js", type='module'),
        Link(rel="preconnect", href="https://fonts.googleapis.com"), Link(rel="preconnect", href="https://fonts.gstatic.com", crossorigin=""),
        Link(rel="stylesheet", href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans:wght@400;500;600;700&family=IBM+Plex+Mono&display=swap"),
        Style("body { font-family: 'IBM Plex Sans', sans-serif; } code, pre { font-family: 'IBM Plex Mono', monospace; }"),
        Script("if(!localStorage.__FRANKEN__) localStorage.__FRANKEN__ = JSON.stringify({mode: 'light'})"))

app = FastHTML(hdrs=hdrs)
rt = app.route

In [ ]:
#| export
# CSS class definitions for reusable components
NAVBAR_CLS = "flex items-center justify-between bg-slate-900 text-white p-4 my-4 rounded-lg shadow-md dark:bg-slate-800"
FOOTER_CLS = "bg-slate-900 text-white rounded-lg p-4 my-4 dark:bg-slate-800"
MAIN_CLS = "w-full max-w-2xl mx-auto px-6 py-8 space-y-8"
CONTAINER_CLS = "w-full max-w-2xl mx-auto"
LINK_CLS = "text-blue-600 hover:underline py-2"
TOGGLE_BTN_CLS = "p-1 hover:scale-110 shadow-none"

def theme_toggle():
    return Button(UkIcon("moon", cls="dark:hidden"), UkIcon("sun", cls="hidden dark:block"),
        _="""on load
                set franken to (localStorage's __FRANKEN__ or '{}') as Object
                if franken's mode is 'dark' then add .dark to <html/> end
                on click
                toggle .dark on <html/>
                set franken to (localStorage's __FRANKEN__ or '{}') as Object
                if the first <html/> matches .dark set franken's mode to 'dark' else set franken's mode to 'light' end
                set localStorage's __FRANKEN__ to franken as JSON""",
        cls=TOGGLE_BTN_CLS, type="button")

def navbar():
    return Div(
        A("Bloggy", href="/"),
        theme_toggle(),
        cls=NAVBAR_CLS
    )

def layout(*content, htmx, title=None):
    if htmx and htmx.request: return (Title(title), *content)
    main = Main(*content, cls=MAIN_CLS, id="main-content")
    return Title(title), Div(cls="flex flex-col min-h-screen")(
        Div(navbar(), cls=f"{CONTAINER_CLS} px-4 sticky top-0 z-50 mt-4"),
        main,
        Footer(Div("Footer Placeholder", cls=FOOTER_CLS), cls=f'{CONTAINER_CLS} px-6 mt-auto mb-6 sticky bottom-0 z-50')
    )

def get_posts(): 
    # recursively discover markdown files in the root folder
    global ROOT_FOLDER
    md_files = list(ROOT_FOLDER.rglob('*.md'))
    posts = []
    for file in md_files:
        # Get the relative path from ROOT_FOLDER
        rel_path = file.relative_to(ROOT_FOLDER).with_suffix('')
        slug = str(rel_path)  # Keep the full path with slashes
        posts.append(Li('- ', A(file.stem, href=f'/posts/{slug}'), cls=LINK_CLS))
    return posts

@rt('/posts/{path:path}')
def post_detail(path: str, htmx):
    # path parameter already includes the full path with slashes
    global ROOT_FOLDER
    file_path = ROOT_FOLDER / f'{path}.md'
    with open(file_path) as f:
        content = render_md(f)
    return layout(
        H1(f"Post: {path}", cls="text-4xl font-bold"),
        content,
        htmx=htmx,
        title=f"{path} - Bloggy",
    )

@rt
def index(htmx):
    return layout(
        H1("Welcome to Bloggy!", cls="text-4xl font-bold"),
        P("This is the home page of Bloggy, your personal blogging platform."),
        Ul(*get_posts()),
        htmx=htmx,
        title="Home - Bloggy",
    )

In [5]:
ROOT_FOLDER = Path('/Users/yeshwanth/Code/Personal/bloggy/demo')
srv = nb_serve(app, port=7860)

In [80]:
srv.should_exit = True
wait_port_free(7860)

In [ ]:
# | hide
import nbdev
nbdev.nbdev_export()